In [1]:
import numpy as np
import NonLinearModels_ante as nlm
import pandas as pd
from tabulate import tabulate
from numpy.random import default_rng

%load_ext autoreload
%autoreload 2

Introduction
============

In this week's problem set, you are asked to investigate how alcohol
abuse affects employment status. Individuals can either be outside of
the labour market, unemployed or employed. This means, that the
dependent variable has three discrete outcomes. We use the multinomial
logit model to express the probability of individuals to be either
outside the labour market, unemployed or employed. This model is a
generalization of the logit model we looked at in Problem set 4 (binary
choice) which only allowed for the dependent variable to have two
discrete outcomes.

In the multinomial logit the probability of observing choice $j$ for
household $i$ given the observed explanatory variables, $x_{i}$ is given
as,
$$
\text{Pr}\left(y_{i}=j\mid x_{i},\beta_j\right)=p_{ij}=\frac{\exp\left(x_{i}^{\prime}\beta_j\right)}{\sum_{m=1}^{J}\exp\left(x_{i}^{\prime}\beta_m\right)},\quad j=0,\dots J \tag{1}
$$

The probabilities lie between 0 and 1 since
$\exp\left(x_{i}^{\prime}\beta_j\right)>0$ and they sum to one over the
J alternatives. The expression,
(1), can be derived from an underlying latent
utility model, where taste shocks follow an extreme value type 1
distribution. The data-generating process of this latent utility model
can be formalized as

$$
\begin{aligned}
y_{i} &= \operatorname*{argmax}_{j=0,\ldots,J} \{u_{ij}\}, &
y_{i} &\in \{0,\ldots,J\}, & (2) \\[1ex] 
u_{ij} &= x_{i}'\beta_j + \varepsilon_{ij}, & 
\varepsilon_{ij} &\sim \text{Gumbel}(0,1), & (3)
\end{aligned}
$$

where the utility, $u_{ij}$, consists of a deterministic part,
$V_{ij}=x_i'\beta_j$, and a stochastic part, $\epsilon_{ij}$. Individual
$i$ chooses the alternative $y_i\in \{1,\ldots,J\}$ that is associated
with the highest level of $u_{ij}$ after having observed the iid extreme
value type 1 distributed shocks, $\epsilon_{ij}$ for
$j\in\{1,\ldots,J\}$. The choice probabilities
(1) can be derived from this underlying utility
model treating the utility shocks, $\epsilon_{ij}$, as unobserved by the
econometrician. The probability of individual $i$ choosing alternative
$j$ is then equal to the probability that the utility, $u_{ij}$, is the
highest among the $J$ alternatives that individual $i$ can choose
between, i.e.,
$p_{ij}=Pr(u_{ij}=\max\{u_{i0},\ldots,u_{iJ}\}\mid x_i,\beta_0,\ldots,\beta_J)$.

Question 1.
-----------

The first task is to finish the code in the function
`sim_data(n, j, \theta)` in the `NonLinearModels` module. This
function simulates a data set with $n$ individuals choosing between $j$ alternatives. The third input is a parameter vector,
$\theta=(\beta_1^\prime,\ldots,\beta_J^\prime)^\prime$ and $J=0$ is the baseline alternative $\beta_0=0$.

The function simulates a $N \times K$ matrix of covariates. These
consist of a constant term and K-1 individual characteristics drawn from a standard normal distribution, i.e.
$x_{i}= (1,x_{i2}, \ldots, x_{iK})'$, with $x_{ik} \sim N(0,1)$.

The choice pattern is simulated in accordance with the data generating
process given by the equations
(2) - (3).

*Note:*
The solution to this is exactly as we simulated data for the clogit last week. The main difference is that we now have beta coefficients for each option (last week we had one column of beta coefficients, today we have (J - 1) columns of beta coefficients.). As mentioned, we use the first option J = 0 as a baseline.

*Programming hing:*
To practically implement this, you need to first initialize the `v` matrix from last time, for example like this: `v = np.zeros((n, j))`. Then you need to loop over the columns of `v`, except for the very first column, and fill these columns with the product from `x @ beta_j`, where `beta_j` is a column of the beta coefficients for that alternative. Again, since the first option is the baseline, `v` should end up with its first column filled with only zeros, and the rest of the columns should be filled with non-zero values.

In [2]:
# Initialize simulation parameters
n = 10000
j = 3

# Initialize true beta coefficients for data generating process.
beta2 = np.array([[-.3, .6]]).T
beta3 = np.array([[.2, -1]]).T
true_beta = np.hstack((beta2, beta3))
theta0 = np.zeros((j - 1, j - 1))

In [3]:
# Simulate some data
sim_result = nlm.sim_data(n, j, true_beta)
y = sim_result.get('y')
x = sim_result.get('x')

IndexError: index 3 is out of bounds for axis 1 with size 3

In [4]:
[print(f'Category {i} has {y[y == i].size} observations.') for i in np.unique(y)] ;

Category 0 has 2959 observations.
Category 1 has 2800 observations.
Category 2 has 4241 observations.


Question 2. 
-----------

The choice probabilities in the multinomial logit model is given by
equation (1). This should be implemented into the mlogit.m
class file such that the function
`choice_prob()` returns choice probabilities, $p_{ij}$ for
$i\in \{0,\ldots,N\}$ and $j\in \{0,\ldots,N\}$. The choice
probabilities are contained in the ($N \times J$) matrix `ccp`. The function should also return the (natural)
$\log$ of the choice probabilities which is used later in the maximum
likelihood estimation. The log transformed choice probabilities, which should be saved in the matrix `logccp`,
are given by
$$
\log\text{Pr}\left(y_{i}=j\mid x_{ij},\beta\right)=\log p_{ij}=x_{i}^{\prime}\beta_j-\log\left(\sum_{m=0}^{J}\exp\left(x_{i}^{\prime}\beta_m\right)\right),
$$

*Programming hint*:
Again, this is done exactly like last week, except that you need a loop for each column of coefficients we have for the choices when you calculate the utility matrix. Remember that the first column in the utility matrix is the baseline, and therefore should be a column with only zero values.

In [5]:
ccp, logccp = nlm.choice_prob(theta0, x)

Question 3. 
-----------

Last time you created the criterion function for the conditional logit estimator. This is identical to the same criterion function for the multinomial logit model, so you need only to copy this from last week.

Now estimate the coefficients based on the simulated data, using maximum likelihood to solve the model. To do this use the `nlm.estimate()` function.

I have written some code that calculates the norm of the gradient at the point of the solution. We will use this later to solve any issues we might have in a real, empirical example. Should this norm be large or small?

In [6]:
sim_result = nlm.estimate(nlm.mlogit, theta0, y, x)

Optimization terminated successfully.
         Current function value: 0.931889
         Iterations: 17
         Function evaluations: 108
         Gradient evaluations: 18


In [7]:
print(true_beta)
nlm.print_table(('y', ['x21', 'x22', 'x31', 'x32']), sim_result)

[[-0.3  0.2]
 [ 0.6 -1. ]]
Results
Dependent variable: y

          beta         se
---  ---------  ---------
x21  -0.296056  0.030534
x22   0.191351  0.0264734
x31   0.549629  0.0317902
x32  -1.03757   0.0318461
In 17 iterations and 108 function evaluations.


In [8]:
def calc_norm(q, theta):
    print(np.linalg.norm(
        np.sum(nlm.centered_grad(q, theta), axis=0)
    ))

In [9]:
q = lambda theta: -(1/n)*np.sum(nlm.mlogit(theta, y, x))
calc_norm(q, sim_result['b_hat'])

4.8494666049065975e-06


Question 4. 
-----------

We are now going to use the data from Terza (2002) aticle, ALCOHOL ABUSE AND EMPLOYMENT: A SECOND LOOK, and estimate the
multinomial logit model. Therefore, we need to first load the data and
then use the proper functions coded previously. 

The persons in this data have three states: 0 if they are out of the work force, 1 if they are unemployed, and 2 if they are employed. Out of the work force will be the baseline for our analysis.

The results we are looking to obtain are shown in the table below, although some of them are rescaled, so you might have too many commas for some coefficients:

``` {.matlab}
                      Unemployed             Employed              
                             
   Alc. abuse:    0.127    [t =   0.59]   -0.153    [t =  -1.10]
   Unemp.rate:    0.458    [t =   0.92]   -0.955    [t =  -2.85]
          Age:    1.618    [t =   2.41]    2.272    [t =   5.49]
  Age-squared:   -2.438    [t =  -2.99]   -3.080    [t =  -6.29]
    Schooling:   -0.092    [t =  -0.39]    0.891    [t =   5.55]
      Married:    0.400    [t =   1.96]    0.709    [t =   5.39]
  Family size:    0.062    [t =   1.20]    0.062    [t =   1.75]
        White:    0.039    [t =   0.23]    0.738    [t =   6.47]
    Excellent:    2.918    [t =   6.41]    3.703    [t =  19.41]
    Very Good:    2.978    [t =   6.52]    3.653    [t =  18.87]
         Good:    2.494    [t =   5.53]    3.000    [t =  16.16]
         Fair:    1.460    [t =   2.99]    1.876    [t =   9.57]
    Northwest:    0.085    [t =   0.36]    0.089    [t =   0.59]
      Midwest:    0.016    [t =   0.08]    0.123    [t =   0.95]
        South:    0.175    [t =   0.87]    0.439    [t =   3.28]
  Center City:   -0.272    [t =  -1.42]   -0.269    [t =  -2.12]
    Other MSA:   -0.092    [t =  -0.47]    0.098    [t =   0.77]
  1st quarter:    0.422    [t =   2.14]   -0.028    [t =  -0.21]
  2nd quarter:   -0.022    [t =  -0.11]   -0.111    [t =  -0.88]
  3rd quarter:   -0.037    [t =  -0.17]   -0.053    [t =  -0.40]
     Constant:   -6.114    [t =  -4.31]   -6.237    [t =  -7.17]
     
     Value of the log-likehood function: -3217.48
```


I have written the code to load the data. Use this data to estimate a multinomial logit model using the `nlm.estimate()` function, and print it out using the `nlm.print_table()` function.

You might get an error that states that the minimizer is unsure whether it found a solution. Reuse the code from above and check the size of the norm at the solution. Remember that you have to redefine `q` as well. Does the norm look fine to you?

In [10]:
data = np.loadtxt('terza.txt')
y = data[:, 0] - 1  # Subtract 1 to make into 0-index
y = y.astype(int)

const = np.ones((y.size, 1))
x = np.hstack((data[:, 1:], const))

y_lab = 'Employed - Unemployed'
x_lab = ['alcohol', 'urate', 'age', 'agesq', 'schooling', 'married', 'famsize', 'white', 'excellent', 'verygood', 'good', 'fair', 'northeast', 'midwest', 'south', 'center', 'other', 'firstq', 'secondq', 'thirdq', 'const']

j = int(y.max())  # Number of alternatives, including 0.

theta0 = np.zeros((x.shape[1], j))

In [11]:
# Use nlm.estimate and nlm.print_table to estimate the model and print it out nicely. You should get coefficients that are close to the table above.

In [12]:
# Calculate the norm of the gradient, you can reuse the function defined above, but you need to redefine the anonymus function q yourself.

Question 5. 
-----------
If the magnitudes of our covariates vary by a lot, our minimizer may sometimes have issues finding a solution to our problem, even though it is very close to a solution.

I have made a copy of `x`, called `x_scale`. Downscale columns 1, 2 and 4 in `x_scale` by 10, and downscale column 3 by 1000. (By downscale I mean that you divide the values by 10 or 1000). Then resolve the model with this rescaled `x` matrix. Is the model now solved? Do the results differ? How does the norm look like now?

To reitterate:
- Rescale the `x` matrix into the new `x_scale` matrix
- Resolve the model using the new `x_scale` matrix
- Print out a table with the new results, do they differ from the previous solution?
- Calculate the norm of the new solution, does it look better?

In [13]:
x_scale = x.copy()
x_scale[:, 1] = x_scale[:, 1]/10
x_scale[:, 2] = x_scale[:, 2]/10
x_scale[:, 3] = x_scale[:, 3]/1000
x_scale[:, 4] = x_scale[:, 4]/10

In [14]:
# Repeat what you did in the previous question. Does the norm look better now?

## Question 6.

Consider a particular individual represented by
the row vector of covariates
$x_0 = (0, 6, 40, 40^2, 8, 0, 2, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1)$
where the first element indicates that the individual does not abuse
alcohol ($x_0[1]= 0$). Note that to facilitate convergence we need to
again rescale some of the covariates. Following the rescale proposed in
the expost code, the vector of covariates will become
$x_0 = (0, 0.6, 4, 1.6, 0.8, 0, 2, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1)$.

The odds ratio measures the probability of a given outcome to
occur relative to the probability of a baseline outcome, conditional on
the covariates. It is defined as:

$$
\begin{aligned}
\frac{Pr(y_i=j|x_i)}{Pr(y_i=1|x_i)} = \frac{\frac{\exp \{ x_{i}^{\prime}\beta_j \}}{ \sum_{l=1}^J \exp \{ x_{i}^{\prime}\beta_l \}}}   {\frac{\exp \{ x_{i}^{\prime}\beta_1 \}}{ \sum_{l=1}^J \exp \{ x_{i}^{\prime}\beta_l \}}} = \exp\{x_i^\prime \beta_j\}
\end{aligned}
$$

The odds ratio of being employed is therefore calculated as
$x_0^\prime \beta_3$, where $\beta_3$ is the vector of parameters
estimated for the third outcome (being employed).

- Calculate the odds ratio of being unemployed to be around 0.593.
- Calculate the odds ratio of being employed to be around 5.742.

How do we interpret these odds ratios?

In [15]:
# Consider some person
x0 = np.array([[0, 6, 40, 40*40, 8, 0, 2, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1]], dtype=np.float).reshape(1, -1)
# Rescale
x0[:, 1] = x0[:, 1]/10
x0[:, 2] = x0[:, 2]/10
x0[:, 3] = x0[:, 3]/1000
x0[:, 4] = x0[:, 4]/10

x1 = x0.copy()
x1[0, 0] = 1.0
betahat = mlogit_result2['b_hat'].reshape(21, 2)

NameError: name 'mlogit_result2' is not defined

In [ ]:
# Calculate the odds ratio of being unemployed/employed for the person x0. These odds ratoi are compared to the baseline out of the workforce. How would you interpret these coefficients?
odds_unemp =  # FILL IN
odds_emp =  # FILL IN
print(f'{odds_unemp.item():.3f}')
print(f'{odds_emp.item():.3f}')

Question 7. 
-----------

The marginal effect of a change in a binary covariate is computed by
subtracting the conditional choice probability under the two possible
values of the binary covariate $k$ (with and without alcohol
abuse): 
$$
\begin{aligned}
\delta^d_j(x_{ik})=Pr(y_i=j|x_{ik}=1)-Pr(y_i=j|x_{ik}=0)
\end{aligned}
$$

- Cacluate the choice probability for person $x0$.
- Cacluate the choice probability for the same person but now with alcholoh abuse set to 1 (x1).
- Subtrace the choice probabilty of x1 from x0.
- You should get values close to `[ 0.0154  0.0211  -0.0365]`

In [ ]:
def marg_effect_discrete(betahat, x0, x1):
    # Calculate the probabilities of x1 and x0, using betahat of the lates model estimation and nlm.choice_prob (remember, it returns two outputs, but we are not interested in the log probabilites, so we often use "_" for something we are not going to use).
    # Then substract the probability vector of x0 from the probability vector of x1.
    prob_x1, _ =  # FILL IN
    prob_x0, _ =  # FILL IN
    return (prob_x1 - prob_x0).flatten()

In [ ]:
me_alc = marg_effect_discrete(mlogit_result2['b_hat'], x0, x1)
print(me_alc)

## Question 8.

The marginal effect for the multinomial logit model with respect to a
continuous covariate $k$ (e.g., schooling) is defined as:
$$
\begin{aligned}
\frac{\partial Pr(y_i=j\mid x_i)}{\partial x_{ik}} = p_{ij} \left(\beta_{jk} - \sum_{l=1}^J p_{il}\beta_{lk}\right),
\end{aligned}
$$

where $p_{ij}$ is the $1 \times 3$ row vector of conditional choice probabilites for our representative person $x_0$, and $\beta_{jk}$ is a $3 \times 1$ vector of the beta coefficients for each choice (remember that the first choice is the baseline, and its coefficient is therefore 0). Note that $\sum_{l=1}^J p_{il}\beta_{lk}$ can be calculated as the dot product between $p_{ij}$ and $\beta_{jk}$.

- Calculate the marginal effect of education using the above equation. $\beta_{jk}$ is given to you.
- You should get values close to `[-0.00940 -0.00632  0.0157]`

In [ ]:
def marg_effect_continous(betahat, betahat_subset, x):
    # Use betahat to calculate the probability vector for person x0.
    # Then use the formula given above to calculate the marginal effect, using the probability vector and the betahat_subset.
    prob, _ = nlm.choice_prob(betahat, x)
    return  # FILL IN

In [ ]:
betahat_educ = np.array(
    [[0.0, betahat[4, 0] / 10, betahat[4, 1] / 10]]
).T
me_educ = marg_effect_continous(betahat, betahat_educ, x0)
print(me_educ)

## Bootstrapping to get standard errors of marginal effects.

Since we are not able to directly calculate variances for the marginal effects, we can use bootstrapping instead. This is done by taking a sample of our data, but with replacement.

This means that we create a new data set as a random draw form our original data set, but we allow that the same observation is drawn multiple times. This new data set has the same number of observations as the original, but will differ sligthly since we now have some duplicates.

We then perform our marginal effect calculations again on this new sample, save the results, and then draw a new random sample from the data (again, with replacement). We continue this process a number of times, and then calculate the standard deviation of the stored marginal effects. This standard deviation is what we will use as standard deviations to the marginal effects calculated on the original data.

In [ ]:
nboot = 10  # Number of bootstraps

# Initialize matrices to store bootstrap results
me_alc_boot = np.zeros((nboot, 3))
me_educ_boot = np.zeros((nboot, 3))

# Set seed for random sampling.
seed = 42
rng = default_rng()

# Stack y and x together, in order to make sure we make a random draw of the same row.
data = np.hstack((y.reshape(-1, 1), x_scale))

In [ ]:
def random_sample(data):
    """Creates a new data set using random draws of the original data set, allowing for replacement. In other words, we allow the same row to be drawn multiple times.
    """
    data_sample = rng.choice(data, n)  # Makes n random draws from data, allowing replacement.

    # We then separate out back to the y vector and x matrix.
    y_sample = data_sample[:, 0]
    y_sample = y_sample.astype(int)  # The y vector is recast into a float vector during the previous np.hstack, but we need it to be int for indexing later. We therefore recast it back into int.
    x_sample = data_sample[:, 1:]
    return y_sample, x_sample

In [ ]:
for i in range(nboot):
    print(f'Bootstrap iteration {i}')
    k = x.shape[1]

    # Get a new random sample each iteration, and resolve the model.
    y_boot, x_boot = random_sample(data)
    boot_result = nlm.estimate(
        nlm.mlogit, 
        mlogit_result2['b_hat'], 
        y_boot, 
        x_boot
    )

    # Save the estimated coefficients, and make the ready for calculating new marginal effects
    betahat_boot = boot_result['b_hat']
    betahat_boot = betahat_boot.reshape(k, -1)
    betahat_boot_educ = np.array(
        [[0.0, betahat_boot[4, 0] / 10, betahat_boot[4, 1] / 10]]
    ).T  # This is the education coefficients.

    # Calculate the marginal effects, and store them for later use.
    me_alc_boot[i] = marg_effect_discrete(betahat_boot, x0, x1)
    me_educ_boot[i] = marg_effect_continous(betahat_boot, betahat_boot_educ, x0)

Here is a function to make a semi-nice table to print the coefficients, and their 95% confidence interval.

In [ ]:
def me_table(me_coeff, me_se, **kwargs):
    table = np.column_stack(
        (me_coeff, me_coeff -1.96*me_se, me_coeff +1.96*me_se)
    )
    print(tabulate(table, **kwargs))

In [ ]:
print("Marginal effect of alcohol on being out of workforce, unemployment or employment:")
me_table(
    me_alc, 
    np.std(me_alc_boot, axis=0),
    headers=['Coeff', '0.5', '0.95'], 
    floatfmt='.4f')

In [ ]:
print("Marginal effect of education on being out of workforce, unemployment or employment:")
me_table(
    me_educ, 
    np.std(me_educ_boot, axis=0),
    headers=['Coeff', '0.5', '0.95'], 
    floatfmt='.4f')